In [9]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

csv_path = "./pokemon.csv" 
img_dir = "./images"    

df = pd.read_csv(csv_path)


df["image_path"] = df["Name"].apply(lambda x: os.path.join(img_dir, f"{x}.png"))

df = df[df["image_path"].apply(os.path.exists)]


df["Type2"] = df["Type2"].fillna("")
df["labels"] = df.apply(lambda x: [x["Type1"]] if x["Type2"] == "" else [x["Type1"], x["Type2"]], axis=1)

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["labels"])

print("클래스(타입):", mlb.classes_)
print("샘플 라벨:", y[0])


train_df, val_df, y_train, y_val = train_test_split(df, y, test_size=0.2, random_state=42)


IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_dataset(df, labels, batch_size=BATCH_SIZE):
    def gen():
        for img_path, label in zip(df["image_path"], labels):
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
            img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            yield img, label

    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32),
            tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.float32)
        )
    )
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = load_dataset(train_df, y_train)
val_ds = load_dataset(val_df, y_val)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(len(mlb.classes_), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=preds)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100
)

sample_img = val_df.iloc[0]["image_path"]
img = tf.keras.preprocessing.image.load_img(sample_img, target_size=IMG_SIZE)
img_arr = tf.keras.preprocessing.image.img_to_array(img)/255.0
img_arr = np.expand_dims(img_arr, axis=0)

pred = model.predict(img_arr)[0]
pred_labels = [mlb.classes_[i] for i, v in enumerate(pred) if v > 0.5]

print("실제 타입:", val_df.iloc[0]["labels"])
print("예측 타입:", pred_labels)


클래스(타입): ['Bug' 'Dark' 'Dragon' 'Electric' 'Fairy' 'Fighting' 'Fire' 'Flying'
 'Ghost' 'Grass' 'Ground' 'Ice' 'Normal' 'Poison' 'Psychic' 'Rock' 'Steel'
 'Water']
샘플 라벨: [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
Epoch 1/100
     21/Unknown 13s 274ms/step - accuracy: 0.0678 - loss: 0.4691

C:\Users\bgs99\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 482ms/step - accuracy: 0.0618 - loss: 0.3844 - val_accuracy: 0.0988 - val_loss: 0.3260
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 371ms/step - accuracy: 0.1329 - loss: 0.3032 - val_accuracy: 0.1481 - val_loss: 0.2837
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 367ms/step - accuracy: 0.1577 - loss: 0.2783 - val_accuracy: 0.1975 - val_loss: 0.2802
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 364ms/step - accuracy: 0.2056 - loss: 0.2629 - val_accuracy: 0.2099 - val_loss: 0.2749
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 353ms/step - accuracy: 0.2117 - loss: 0.2525 - val_accuracy: 0.2284 - val_loss: 0.2725
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 375ms/step - accuracy: 0.2535 - loss: 0.2419 - val_accuracy: 0.2222 - val_loss: 0.2698
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.2720 - loss: 0.2332 - val_accuracy: 0.2222 - val_loss: 0.2683
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 381ms/step - accuracy: 0.2875 - loss: 0.2238 - val_accuracy: 0.2284 